1 clean data


2 k fold for 3 models


3 gridsearchcv 3 models


4 add k fold after getting best parametres from gridsearchcv


5 smote

In [455]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NeighborhoodComponentsAnalysis,KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.svm import SVC

In [456]:
data = pd.read_csv('hcvdat0.csv')

In [457]:
values={'0=Blood Donor':'BloodDonor', '0s=suspect Blood Donor':'suspect Blood Donor', '1=Hepatitis':'Hepatitis', '2=Fibrosis':'Fibrosis', '3=Cirrhosis':'Cirrhosis'}

In [458]:

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
##Encode target labels of cat with value between 0 and n_classes-1.
category = le.fit_transform(data.Category)

In [459]:

##Encode target labels of sex  with value between 0 and n_classes-1.
sex = le.fit_transform(data.Sex)

In [460]:
data['sex'] = sex 
data['category'] = category
data = data.drop('Sex',axis=1)
data = data.drop('Category',axis=1)
##drop labels of unnaled
data = data.drop('Unnamed: 0',axis=True)

In [461]:
##Fill null value with mean
MeanValues0 = {
          'ALB':data.loc[data['category'] == 0].ALB.mean() , 
          'ALP': data.loc[data['category'] == 0].ALP.mean(), 
          'ALT': data.loc[data['category'] == 0].ALT.mean(), 
          'CHOL': data.loc[data['category'] == 0].CHOL.mean(),
          'PROT': data.loc[data['category'] == 0].PROT.mean(),           
         }
MeanValues1 = {
          'ALB':data.loc[data['category'] == 1].ALB.mean() , 
          'ALP': data.loc[data['category'] == 1].ALP.mean(), 
          'ALT': data.loc[data['category'] == 1].ALT.mean(), 
          'CHOL': data.loc[data['category'] == 1].CHOL.mean(),
          'PROT': data.loc[data['category'] == 1].PROT.mean(),           
         }
MeanValues2 = {
          'ALB':data.loc[data['category'] == 2].ALB.mean() , 
          'ALP': data.loc[data['category'] == 2].ALP.mean(), 
          'ALT': data.loc[data['category'] == 2].ALT.mean(), 
          'CHOL': data.loc[data['category'] == 2].CHOL.mean(),
          'PROT': data.loc[data['category'] == 2].PROT.mean(),           
         }
MeanValues3 = {
          'ALB':data.loc[data['category'] == 3].ALB.mean() , 
          'ALP': data.loc[data['category'] == 3].ALP.mean(), 
          'ALT': data.loc[data['category'] == 3].ALT.mean(), 
          'CHOL': data.loc[data['category'] == 3].CHOL.mean(),
          'PROT': data.loc[data['category'] == 3].PROT.mean(),           
         }
MeanValues4 = {
          'ALB':data.loc[data['category'] == 4].ALB.mean() , 
          'ALP': data.loc[data['category'] == 4].ALP.mean(), 
          'ALT': data.loc[data['category'] == 4].ALT.mean(), 
          'CHOL': data.loc[data['category'] == 4].CHOL.mean(),
          'PROT': data.loc[data['category'] == 4].PROT.mean(),           
         }

In [462]:

data.isnull().sum()

Age          0
ALB          1
ALP         18
ALT          1
AST          0
BIL          0
CHE          0
CHOL        10
CREA         0
GGT          0
PROT         1
sex          0
category     0
dtype: int64

In [463]:
data.loc[data['category'] == 0]=data.loc[data['category'] == 0].fillna(value = MeanValues0)
data.loc[data['category'] == 1]=data.loc[data['category'] == 1].fillna(value = MeanValues1)
data.loc[data['category'] == 2]=data.loc[data['category'] == 2].fillna(value = MeanValues2)
data.loc[data['category'] == 3]=data.loc[data['category'] == 3].fillna(value = MeanValues3)
data.loc[data['category'] == 4]=data.loc[data['category'] == 4].fillna(value = MeanValues4)

In [464]:

data.isnull().sum()

Age         0
ALB         0
ALP         0
ALT         0
AST         0
BIL         0
CHE         0
CHOL        0
CREA        0
GGT         0
PROT        0
sex         0
category    0
dtype: int64

In [467]:

X = data.iloc[:, 0:12]
Y = data.iloc[:,-1]

In [468]:

Y

0      0
1      0
2      0
3      0
4      0
      ..
610    4
611    4
612    4
613    4
614    4
Name: category, Length: 615, dtype: int64

In [469]:
X

,Age,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,sex
0,32,38.5,52.500000,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0,1
1,32,38.5,70.300000,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5,1
2,32,46.9,74.700000,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3,1
3,32,43.2,52.000000,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7,1
4,32,39.2,74.100000,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7,1
...,...,...,...,...,...,...,...,...,...,...,...,...
610,62,32.0,416.600000,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5,0
611,64,24.0,102.800000,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3,0
612,64,29.0,87.300000,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0,0
613,46,33.0,93.220833,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0,0


In [470]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2,stratify=Y
)

# 2_  k-fold for 3 models

In [419]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
cv =KFold(n_splits=9, random_state=None, shuffle=False)
scores=[]
for train_index, test_index in cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    modelsvc.fit(X_train, y_train) 
    scores.append(modelsvc.score(X_train, y_train))
    predictions = modelsvc.predict(X_test) 
    cm= confusion_matrix(y_test, predictions)
    print(classification_report(y_test, predictions))
    print(cm)
print(np.mean(scores))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

[[69]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        69
           2       0.00      0.00      0.00         0

    accuracy                           0.99        69
   macro avg       0.50      0.49      0.50        69
weighted avg       1.00      0.99      0.99        69

[[68  1]
 [ 0  0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

[[69]]
              precision    recall  f1-score   support

           0       1.00      1.00      

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: Und

In [418]:
cv =KFold(n_splits=9, random_state=None, shuffle=False)
scores=[]

for train_index, test_index in cv.split(X):
    print("______________________________________________________________________________")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    ModelKNN.fit(X_train, y_train) 
    scores.append(ModelKNN.score(X_train, y_train))
    predictions = ModelKNN.predict(X_test) 
    cm= confusion_matrix(y_test, predictions)
    print(classification_report(y_test, predictions))
    print(cm)
print(np.mean(scores))

______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        69
           2       0.00      0.00      0.00         0

    accuracy                           0.97        69
   macro avg       0.50      0.49      0.49        69
weighted avg       1.00      0.97      0.99        69

[[67  2]
 [ 0  0]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        69
           3       0.00      0.00      0.00         0

    accuracy                           0.99        69
   macro avg       0.50      0.49      0.50        69
weighted avg       1.00      0.99      0.99        69

[[68  1]
 [ 0  0]]
______________________________________________________________________________


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

[[69]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        68

    accuracy                           1.00        68
   macro avg       1.00      1.00      1.00        68
weighted avg       1.00      1.00      1.00        68

[[68]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        68

    accuracy                           1.00        68
   macro avg       1.00      1.00      1.00        68
weighted avg       1.00      1.00      1.00        68

[[68]]
__

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1

In [417]:

cv =KFold(n_splits=9, random_state=None, shuffle=False)
scores=[]
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
for train_index, test_index in cv.split(X):
    print("______________________________________________________________________________")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    modelrf.fit(X_train, y_train) 
    scores.append(modelrf.score(X_train, y_train))
    predictions = modelrf.predict(X_test) 
    cm= confusion_matrix(y_test, predictions)
    print(classification_report(y_test, predictions))
    print(cm)
print(np.mean(scores))

______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

[[69]]
______________________________________________________________________________


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.99      0.99        69
           3       0.00      0.00      0.00         0

    accuracy                           0.99        69
   macro avg       0.50      0.49      0.50        69
weighted avg       1.00      0.99      0.99        69

[[68  1]
 [ 0  0]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

[[69]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        68

    accuracy                           1.00        68
   macro avg       1.00      1.00      1.00        6

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.87      0.98      0.92        54
           1       0.00      0.00      0.00         7
           2       1.00      0.14      0.25         7
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.79        68
   macro avg       0.37      0.22      0.23        68
weighted avg       0.79      0.79      0.76        68

[[53  0  0  0  1]
 [ 5  0  0  0  2]
 [ 3  0  1  3  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       0.0
           2       0.00      0.00      0.00      17.0
           3       0.00      0.00      0.00      21.0
           4       0.00      0.00      0.00      30.0

    accuracy   

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 3_ gridsearch pour  les 3methode

In [471]:
from sklearn.pipeline import Pipeline
steps = [('scaler', StandardScaler()), ('SVM', SVC())]
pipeline_1 = Pipeline(steps)
pipeline_1

Pipeline(steps=[('scaler', StandardScaler()), ('SVM', SVC())])

In [472]:
parameteres = {'SVM__C':[0.001,0.1,10,100,10e5], 'SVM__gamma':[0.1,0.01]}

In [473]:
grid = GridSearchCV(pipeline_1, param_grid=parameteres, cv=5)
grid.fit(X_train, y_train)
grid

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('SVM', SVC())]),
             param_grid={'SVM__C': [0.001, 0.1, 10, 100, 1000000.0],
                         'SVM__gamma': [0.1, 0.01]})

In [474]:
print(grid.best_params_)
y_true, y_pred2 = y_test, grid.predict(X_test)
print(classification_report(y_true, y_pred2))
cm= confusion_matrix(y_test, y_pred2)
cm

{'SVM__C': 100, 'SVM__gamma': 0.01}
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       107
           1       0.00      0.00      0.00         1
           2       0.25      0.40      0.31         5
           3       0.33      0.25      0.29         4
           4       0.71      0.83      0.77         6

    accuracy                           0.91       123
   macro avg       0.46      0.49      0.47       123
weighted avg       0.92      0.91      0.91       123



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[104,   0,   2,   0,   1],
       [  1,   0,   0,   0,   0],
       [  0,   0,   2,   2,   1],
       [  0,   0,   3,   1,   0],
       [  0,   0,   1,   0,   5]])

In [479]:
pipeline_2 = Pipeline([('scaler', StandardScaler()),('nca', NeighborhoodComponentsAnalysis()), ('knn', KNeighborsClassifier())])
pipeline_2
#parameters
parameteres_knn = {
    
    'knn__n_neighbors':[5,3,8,1],
    'knn__algorithm':['auto','ball_tree','kd_tree','brute'],
    'knn__leaf_size':[5,20,25,30,50]
    
}
# Grdisearch
grid = GridSearchCV(pipeline_2, param_grid=parameteres_knn, cv=5)
grid.fit(X_train, y_train)
grid
print(grid.best_params_)
y_true, y_pred2 = y_test, grid.predict(X_test)

{'knn__algorithm': 'auto', 'knn__leaf_size': 5, 'knn__n_neighbors': 5}


In [480]:
print(classification_report(y_true, y_pred2))
cm= confusion_matrix(y_test, y_pred2)
cm

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       107
           1       0.00      0.00      0.00         1
           2       0.67      0.40      0.50         5
           3       0.75      0.75      0.75         4
           4       0.83      0.83      0.83         6

    accuracy                           0.95       123
   macro avg       0.64      0.60      0.61       123
weighted avg       0.94      0.95      0.94       123



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[107,   0,   0,   0,   0],
       [  1,   0,   0,   0,   0],
       [  2,   0,   2,   0,   1],
       [  0,   0,   1,   3,   0],
       [  0,   0,   0,   1,   5]])

In [495]:
rf = RandomForestClassifier()
pipeline_3 = Pipeline([('scaler', StandardScaler()), ('rf', rf)])
pipeline_3
parameteres_rf = {
    
    'rf__n_estimators':[20,50,80,150,200],
    'rf__criterion':['entropy','gini'],
    'rf__max_depth':[5,20,25,30,50,100] 
    
}
# Grdisearch
grid = GridSearchCV(pipeline_3, param_grid=parameteres_rf, cv=5)
grid.fit(X_train, y_train)
grid
print(grid.best_params_)

{'rf__criterion': 'entropy', 'rf__max_depth': 30, 'rf__n_estimators': 20}


In [483]:
y_true, y_pred2 = y_test, grid.predict(X_test)
print(classification_report(y_true, y_pred2))
cm= confusion_matrix(y_test, y_pred2)
cm

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       107
           1       0.00      0.00      0.00         1
           2       0.50      0.40      0.44         5
           3       1.00      0.25      0.40         4
           4       0.83      0.83      0.83         6

    accuracy                           0.93       123
   macro avg       0.66      0.50      0.53       123
weighted avg       0.92      0.93      0.92       123



/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[107,   0,   0,   0,   0],
       [  1,   0,   0,   0,   0],
       [  2,   0,   2,   0,   1],
       [  1,   0,   2,   1,   0],
       [  1,   0,   0,   0,   5]])

# 4_ apres le evalution des parametre avec  gridsearchcv on re essaye  les 3 methodes avec k-fold

In [312]:
modelsvc2 = SVC(C=10, gamma=0.01)
cv =KFold(n_splits=9, random_state=None, shuffle=False)
scores=[]
from sklearn.preprocessing import MinMaxScaler
for train_index, test_index in cv.split(X):
    print("______________________________________________________________________________")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    modelsvc2.fit(X_train, y_train) 
    scores.append(modelsvc2.score(X_train, y_train))
    predictions = modelsvc2.predict(X_test) 
    cm= confusion_matrix(y_test, predictions)
    print(classification_report(y_test, predictions))
    print(cm)
print(np.mean(scores))

______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

[[69]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

[[69]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1

In [313]:
knn2 = KNeighborsClassifier(n_neighbors=3,leaf_size=5)
cv =KFold(n_splits=9, random_state=None, shuffle=False)
scores=[]
from sklearn.preprocessing import MinMaxScaler
for train_index, test_index in cv.split(X):
    print("______________________________________________________________________________")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    knn2.fit(X_train, y_train) 
    scores.append(knn2.score(X_train, y_train))
    predictions = knn2.predict(X_test) 
    cm= confusion_matrix(y_test, predictions)
    print(classification_report(y_test, predictions))
    print(cm)
print(np.mean(scores))

______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        69
           2       0.00      0.00      0.00         0

    accuracy                           0.97        69
   macro avg       0.50      0.49      0.49        69
weighted avg       1.00      0.97      0.99        69

[[67  2]
 [ 0  0]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        69
           3       0.00      0.00      0.00         0

    accuracy                           0.99        69
   macro avg       0.50      0.49      0.50        69
weighted avg       1.00      0.99      0.99        69

[[68  1]
 [ 0  0]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0    

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        68

    accuracy                           1.00        68
   macro avg       1.00      1.00      1.00        68
weighted avg       1.00      1.00      1.00        68

[[68]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        68
           1       0.00      0.00      0.00         0

    accuracy                           0.99        68
   macro avg       0.50      0.49      0.50        68
weighted avg       1.00      0.99      0.99        68

[[67  1]
 [ 0  0]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        68
           2       0.00      0.00      0.00         0

    accuracy                           0.99        6

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMe

              precision    recall  f1-score   support

           0       0.82      0.98      0.89        54
           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00         0

    accuracy                           0.78        68
   macro avg       0.20      0.25      0.22        68
weighted avg       0.65      0.78      0.71        68

[[53  0  0  1]
 [ 5  0  0  2]
 [ 7  0  0  0]
 [ 0  0  0  0]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           2       0.33      0.06      0.10        17
           3       0.00      0.00      0.00        21
           4       0.00      0.00      0.00        30

    accuracy                           0.01        68
   macro avg       0.08      0.01      0.03        68
weighted avg       0.08      0.01      0.03  

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [314]:
rf3 = RandomForestClassifier(max_depth=30,n_estimators=20)
cv =KFold(n_splits=9, random_state=None, shuffle=False)
scores=[]
from sklearn.preprocessing import MinMaxScaler
for train_index, test_index in cv.split(X):
    print("______________________________________________________________________________")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    rf3.fit(X_train, y_train) 
    scores.append(rf3.score(X_train, y_train))
    predictions = rf3.predict(X_test) 
    cm= confusion_matrix(y_test, predictions)
    print(classification_report(y_test, predictions))
    print(cm)
print(np.mean(scores))

______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

[[69]]
______________________________________________________________________________


/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      0.99      0.99        69
           3       0.00      0.00      0.00         0

    accuracy                           0.99        69
   macro avg       0.50      0.49      0.50        69
weighted avg       1.00      0.99      0.99        69

[[68  1]
 [ 0  0]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69

    accuracy                           1.00        69
   macro avg       1.00      1.00      1.00        69
weighted avg       1.00      1.00      1.00        69

[[69]]
______________________________________________________________________________
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        68

    accuracy                           1.00        68
   macro avg       1.00      1.00      1.00        6

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: Und

# 5_ Smote 

In [116]:
conda install imbalanced-learn -c conda-forge 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.0
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - imbalanced-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    certifi-2020.12.5          |   py38h578d9bd_0         143 KB  conda-forge
    imbalanced-learn-0.7.0     |             py_1          97 KB  conda-forge
    openssl-1.1.1i             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

  imbalanced-learn   conda-forge/noarch::imbalanced-learn-0.7.0-

In [484]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()

X,Y = oversample.fit_resample(X,Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)

In [485]:
modelsvc3 = SVC()
modelsvc3.fit(X_train, y_train) 
predictions = modelsvc3.predict(X_test) 
print(classification_report(y_test, predictions))
cm= confusion_matrix(y_test, predictions)
cm


              precision    recall  f1-score   support

           0       0.96      0.92      0.94       119
           1       0.98      1.00      0.99       128
           2       0.91      0.89      0.90       154
           3       0.86      0.92      0.89       131
           4       0.99      0.96      0.98       135

    accuracy                           0.94       667
   macro avg       0.94      0.94      0.94       667
weighted avg       0.94      0.94      0.94       667



array([[110,   2,   2,   4,   1],
       [  0, 128,   0,   0,   0],
       [  5,   0, 137,  12,   0],
       [  0,   0,  11, 120,   0],
       [  0,   1,   1,   3, 130]])

In [486]:
knn3 = KNeighborsClassifier(n_neighbors=5,leaf_size=5)
knn3.fit(X_train, y_train)
y_pred =knn3.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[109   3   3   1   3]
 [  0 127   0   0   1]
 [  0   0 151   3   0]
 [  0   0   0 131   0]
 [  1   3   1   0 130]]
              precision    recall  f1-score   support

           0       0.99      0.92      0.95       119
           1       0.95      0.99      0.97       128
           2       0.97      0.98      0.98       154
           3       0.97      1.00      0.98       131
           4       0.97      0.96      0.97       135

    accuracy                           0.97       667
   macro avg       0.97      0.97      0.97       667
weighted avg       0.97      0.97      0.97       667



In [488]:
modelrf = RandomForestClassifier(max_depth=30,n_estimators=20)
modelrf.fit(X_train, y_train)
y_pred = modelrf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[115   1   2   1   0]
 [  0 128   0   0   0]
 [  0   0 154   0   0]
 [  0   0   0 131   0]
 [  0   0   0   0 135]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       119
           1       0.99      1.00      1.00       128
           2       0.99      1.00      0.99       154
           3       0.99      1.00      1.00       131
           4       1.00      1.00      1.00       135

    accuracy                           0.99       667
   macro avg       0.99      0.99      0.99       667
weighted avg       0.99      0.99      0.99       667



# EXAMPLE

In [489]:
X = data.iloc[:, 0:12]
Y= data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2
)

In [490]:
RandomForestClassifier(max_depth=30,n_estimators=20)
model.fit(X_train,y_train)
df={
    'Age':23,
    'ALB':47,
    'ALP':19.1,
    'ALT':38.9,
    'AST':164.2,
    'BIL':17,
    'CHE':7.09,
    'CHOL': 3.2,
    'CREA':79.3,
    'GGT':90.4,
    'PROT':70.1,
    'sex':0
}
df = pd.DataFrame(data=df,columns=['Age','ALB','ALP','ALT','AST','BIL','CHE','CHOL','CREA','GGT','PROT','sex'], index=[0])
df
x = int(model.predict(df))
if x == 0:
    print("Blood Donor")
elif x == 1:
    print("Suspect Blood Donor")
elif x == 2:
    print("Hepatitis")
elif x == 3:
    print("Fibrosis")
elif x == 3:
    print("Cirrhosis")
else :
    print("err")

Hepatitis


# 